In [3]:
#!/usr/bin/python3

In [97]:
# to use 
# do <pip install Faker> in your activated dev env 

In [46]:
from faker import Faker
import random_address
import pandas as pd 
import datetime


In [ ]:
# data cleaner 

In [47]:

first_names = []
last_names = []
jobs = []
credit_scores = []
address = []
year1 = []
year2 = []
year3 = []
dob = []
ssn = []
current_debt = []

faker = Faker()

for i in range(30):
    first_names.append(faker.first_name())
    last_names.append(faker.last_name())
    jobs.append((faker.job()))
    credit_scores.append(faker.random_int(585, 790))
    address.append(faker.address())
    year1.append(faker.random_int(98000, 220000))
    year2.append(faker.random_int(120000, 370000))
    year3.append(faker.random_int(135000, 450000))
    ssn.append(str(faker.random_int(135, 654)) + '-' + str(faker.random_int(23, 88)) + '-' + str(faker.random_int(2234, 7856)))
    current_debt.append(faker.random_int(4500, 62000))
    

In [87]:
# create an empty dob list 
dob = []

start_date = datetime.date(year=1974, month=1, day=1)

for i in range(20):
    # fake.date_between(start_date='today', end_date='+30y')
    dob.append(str(faker.date_between(start_date=start_date, end_date='-32y')))
    
for i in range(10):
    dob.append(str(faker.date_between(start_date=start_date, end_date='-42y')))
for i in range(10):
    dob.append(str(faker.date_between(start_date=start_date, end_date='-45y')))

In [88]:
dob

['1988-05-06',
 '1990-06-24',
 '1976-12-21',
 '1976-03-07',
 '1990-07-21',
 '1979-08-17',
 '1983-01-09',
 '1975-01-14',
 '1983-10-15',
 '1986-12-15',
 '1974-04-24',
 '1981-10-28',
 '1985-05-19',
 '1989-02-01',
 '1976-06-10',
 '1983-11-03',
 '1977-08-29',
 '1974-11-08',
 '1980-12-23',
 '1982-04-05',
 '1974-10-17',
 '1977-10-20',
 '1974-07-30',
 '1974-03-23',
 '1979-10-19',
 '1976-05-28',
 '1978-10-23',
 '1979-01-13',
 '1977-08-05',
 '1977-03-24',
 '1977-01-11',
 '1975-02-08',
 '1975-02-09',
 '1976-07-11',
 '1975-02-19',
 '1974-04-13',
 '1975-08-19',
 '1975-02-18',
 '1977-04-09',
 '1977-05-27']

In [89]:

df = pd.DataFrame(list(zip(first_names, last_names, jobs, credit_scores, address, year1, year2, year3, dob, ssn, current_debt)),
               columns =['first_name', 'last_names', 'jobs', 'credit_scores', 'address', 'year1', 'year2', 'year3', \
                        'dob', 'ssn', 'current_debt'])

In [ ]:
# convert the dob column to datetime object 
df['dob'] = pd.to_datetime(df['dob'])

# subtract 2022 from the year(dob) to get the age of the customer
df['age'] =  df['dob'].apply(lambda x: 2022 - int(x.year))

In [91]:
df.dtypes

first_name       object
last_names       object
jobs             object
credit_scores     int64
address          object
year1             int64
year2             int64
year3             int64
dob              object
ssn              object
current_debt      int64
dtype: object

In [98]:
df.to_csv('customer_data.csv', index=False)

In [118]:
df.head()

,first_name,last_names,jobs,credit_scores,address,year1,year2,year3,dob,ssn,current_debt,age
0,Kim,Bruce,Textile designer,758,"455 Tina Mountain Suite 651\nPort Patricia, MS...",157529,238439,265389,1988-05-06,246-76-6184,37284,34
1,Elizabeth,Palmer,Personal assistant,628,"5357 Kim Ports\nMunozshire, AL 64015",189858,125205,155035,1990-06-24,580-56-7467,15993,32
2,William,Morrison,Human resources officer,654,"36654 Gutierrez Locks Suite 107\nWrighthaven, ...",144727,228111,191532,1976-12-21,649-42-4801,47341,46
3,Charles,Daniel,"Development worker, community",590,"451 Holly Tunnel\nSouth Bill, KS 27892",140893,147183,219216,1976-03-07,604-30-5501,43481,46
4,Larry,Jones,"Designer, exhibition/display",594,"42414 Janet Lights\nWilsonhaven, NJ 28200",129632,258099,188854,1990-07-21,595-36-4379,46826,32


In [123]:
new_strng = '455 Tina Mountain Suite 651\nPort Patricia' 
new_strng.replace('\n', ' ')

'455 Tina Mountain Suite 651 Port Patricia'

In [124]:
def remove_newline(strng):
    return strng.replace('\n', ' ')

In [125]:
df['address'] = df['address'].apply(remove_newline)

In [126]:
df.head(4)

,first_name,last_names,jobs,credit_scores,address,year1,year2,year3,dob,ssn,current_debt,age
0,Kim,Bruce,Textile designer,758,"455 Tina Mountain Suite 651 Port Patricia, MS ...",157529,238439,265389,1988-05-06,246-76-6184,37284,34
1,Elizabeth,Palmer,Personal assistant,628,"5357 Kim Ports Munozshire, AL 64015",189858,125205,155035,1990-06-24,580-56-7467,15993,32
2,William,Morrison,Human resources officer,654,"36654 Gutierrez Locks Suite 107 Wrighthaven, M...",144727,228111,191532,1976-12-21,649-42-4801,47341,46
3,Charles,Daniel,"Development worker, community",590,"451 Holly Tunnel South Bill, KS 27892",140893,147183,219216,1976-03-07,604-30-5501,43481,46


datetime.date(1993, 5, 21)

In [127]:
df_kaggle = pd.read_csv("/Users/patrickrue/Downloads/nyc_housing_data_2013_2019.csv")

In [128]:
df_kaggle.head(3)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,02 TWO FAMILY HOMES,746 EAST 6 STREET,10009.0,2134.0,3542.0,1899.0,1800000,2003-01-22 00:00:00
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,316 EAST 3 STREET,10009.0,5746.0,2700.0,1900.0,0,2003-12-18 00:00:00
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,316 EAST 3 STREET,10009.0,5746.0,2700.0,1900.0,0,2003-12-18 00:00:00


In [129]:
df_kaggle.dtypes

BOROUGH                      int64
NEIGHBORHOOD                object
BUILDING CLASS CATEGORY     object
ADDRESS                     object
ZIP CODE                   float64
LAND SQUARE FEET           float64
GROSS SQUARE FEET          float64
YEAR BUILT                 float64
SALE PRICE                   int64
SALE DATE                   object
dtype: object

In [131]:
# convert ADDRESS, ZIP CODE, LAND SQUARE FEET, GROSS SQUARE FEET
YEAR BUILT
df_kaggle['ZIP CODE'] = pd.to_numeric(df_kaggle['ZIP CODE'], errors='coerce').astype('Int64')

In [134]:
df_kaggle['LAND SQUARE FEET'] = pd.to_numeric(df_kaggle['LAND SQUARE FEET'], errors='coerce').astype('Int64')

In [135]:
df_kaggle['GROSS SQUARE FEET'] = pd.to_numeric(df_kaggle['GROSS SQUARE FEET'], errors='coerce').astype('Int64')

In [136]:
df_kaggle.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,02 TWO FAMILY HOMES,746 EAST 6 STREET,10009,2134,3542,1899.0,1800000,2003-01-22 00:00:00
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,316 EAST 3 STREET,10009,5746,2700,1900.0,0,2003-12-18 00:00:00
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,316 EAST 3 STREET,10009,5746,2700,1900.0,0,2003-12-18 00:00:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,125 AVENUE D,10009,2185,5725,1910.0,426000,2003-10-23 00:00:00
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,610 EAST 9 STREET,10009,2543,11568,1910.0,0,2003-02-28 00:00:00


In [137]:
# remove any houses where sales price is less than 300000
df_kaggle_new = df_kaggle[df_kaggle['SALE PRICE'] > 300000]

In [138]:
df_kaggle_new.head(4)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,ADDRESS,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,02 TWO FAMILY HOMES,746 EAST 6 STREET,10009,2134,3542,1899.0,1800000,2003-01-22 00:00:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,125 AVENUE D,10009,2185,5725,1910.0,426000,2003-10-23 00:00:00
11,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,14 AVENUE A,10009,2377,8722,1900.0,1600000,2003-09-08 00:00:00
12,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,536 EAST 6 STREET,10009,1957,6200,1920.0,1060000,2003-04-30 00:00:00


In [ ]:
# 1 = MANHATTAN
#  2 = BRONX
#  3 = BROOKLYN
#  4 = QUEENS
#  5 = STATEN ISLAND 